# Section 5: Use the sampler primitive

## TASK 5.1: Set sampler primitive options such as dynamical decoupling

1. Which snippet correctly transpiles a parameterized circuit for a chosen backend and produces an ISA circuit before using Estimator V2?

a. 
```python
from qiskit.transpiler import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circ = pm.run(circ)
```
b. 
```python
isa_circ = circ  # QPUs accept any gates; transpilation is optional
```
c. 
```python
from qiskit import transpile
isa_circ = transpile(circ)  # omit backend
```
d. 
```python
pm = generate_preset_pass_manager(backend=None)
isa_circ = pm.run(circ.decompose())
```


:::{dropdown}answer 
The answer is a. 

You must generate a preset pass manager with a concrete `backend` and run it to obtain an ISA circuit; this matches the example that uses `generate_preset_pass_manager(optimization_level=1, backend=backend)` followed by `pm.run(circuit)`. This step ensures the circuit uses only instructions supported by the target device prior to invoking primitives.
:::


2. After transpiling a circuit for Estimator, how do you correctly align the observable to the transpiled circuit layout?

a. 
```python
isa_obs = observable.apply_layout(isa_circ.layout)
```
b. 
```python
isa_obs = observable.transpile_like(isa_circ)
```
c. 
```python
isa_obs = observable.bind_parameters(isa_circ.parameters)
```
d. 
```python
isa_obs = observable.measure_all(inplace=True)
```


:::{dropdown}answer 
The answer is a. 

The documented workflow applies the transpiled circuit’s layout to the observable via `observable.apply_layout(isa_circuit.layout)` so that qubit indices match the ISA circuit fed to Estimator.
:::


3. Which call correctly submits work to Estimator V2 using the Primitive Unified Bloc (PUB) tuple?

a. 
```python
job = estimator.run([(isa_circ, isa_obs, param_values)])
```
b. 
```python
job = estimator.run([(isa_obs, isa_circ, param_values)])
```
c. 
```python
job = estimator.run([(isa_circ, param_values)])
```
d. 
```python
job = estimator.run((isa_circ, isa_obs, param_values))
```


:::{dropdown}answer 
The answer is a. 

The Estimator expects a sequence of PUB tuples in the form `(circuit, observable, parameter_values)`; the example shows `estimator.run([(isa_circuit, isa_observable, param_values)])`.
:::


4. For Sampler V2, which preparation is correct before transpilation and submission?

a. 
```python
circ.measure_all()
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circ = pm.run(circ)
job = sampler.run([(isa_circ, param_values)])
```
b. 
```python
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa_circ = pm.run(circ)  # no measurements needed
job = sampler.run([(isa_circ,)])
```
c. 
```python
job = sampler.run([(circ,)])
```
d. 
```python
isa_circ = circ.remove_final_measurements(inplace=True)
job = sampler.run([(isa_circ,)])
```


:::{dropdown}answer 
The answer is a. 

The sampler example measures the circuit (`measure_all()`), transpiles to ISA with a preset pass manager for the chosen backend, and submits `[(isa_circuit, param_values)]` as the PUB.
:::


5. Which initialization correctly selects the execution mode consistent with the examples?

a. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode=backend)
```
b. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode="session")  # string literal
```
c. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator()  # mode is mandatory
```
d. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode="batch").run(backend)
```


:::{dropdown}answer 
The answer is a. 

The guide initializes Estimator/Sampler with `mode=backend` to select job execution mode on a specific backend. Other forms shown are not aligned with the documented initialization pattern.
:::


6. How can you request a backend that supports fractional gates per the guide?

a. 
```python
service = QiskitRuntimeService()
backend = service.least_busy(use_fractional_gates=True)
```
b. 
```python
service = QiskitRuntimeService()
backend = service.get_backend("fractional-gates")
```
c. 
```python
service = QiskitRuntimeService()
backend = service.least_busy(fractional=True)
```
d. 
```python
service = QiskitRuntimeService()
backend = QiskitRuntimeService(use_fractional_gates=True)
```


:::{dropdown}answer 
The answer is a. 

The page states to set `use_fractional_gates=True` when requesting a backend from `QiskitRuntimeService`, for example `service.least_busy(use_fractional_gates=True)`.
:::


7. Which code path best reflects a minimal, correct optimization/transpilation workflow before running primitives?

a. Create circuit → `generate_preset_pass_manager(optimization_level=1, backend)` → `pm.run(circuit)` → align observable with `apply_layout` → `Estimator.run([...])`<br>
b. Create circuit → call `Estimator.run(circuit)` and let the service infer gates automatically<br>
c. Create circuit → decompose to basis gates locally without a backend → `Estimator.run([...])`<br>
d. Create circuit → add barriers to prevent changes → `Estimator.run([...])`<br>


:::{dropdown}answer 
The answer is a. 

The documented steps show generating a preset pass manager with a specified backend, running it to obtain an ISA circuit, aligning the observable, and then submitting PUBs to the primitive.
:::


8. Which code correctly enables dynamical decoupling (DD) in Sampler V2 and selects the XpXm sequence?

a. 
```
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
```

b. 
```
sampler.options.resilience.zne_mitigation = True
sampler.options.resilience.zne.extrapolator = "exponential"
```

c. 
```
sampler.options.twirling.enable_gates = True
sampler.options.twirling.sequence_type = "XpXm"
```

d. 
```
sampler.set_options(dynamical_decoupling=True, sequence_type="XpXm")
```


:::{dropdown}answer 
The answer is a. 

Sampler V2 exposes DD via `sampler.options.dynamical_decoupling`. Set `enable=True` and choose a valid `sequence_type` such as `"XpXm"`. Error mitigation like ZNE belongs to the Estimator, not Sampler. 
:::


2. Select all valid values for `sampler.options.dynamical_decoupling.sequence_type`. (Multiple answers)

a. "XX"<br>
b. "XpXm"<br>
c. "XY4"<br>
d. "XZXZ"<br>


:::{dropdown}answer 
The answer is a, b, c. 

Valid DD sequence types are `"XX"`, `"XpXm"`, and `"XY4"`. `"XZXZ"` is not a supported built‑in option. 
:::


3. When is enabling dynamical decoupling most likely to help for Sampler runs on hardware?

a. When circuits contain idle gaps on some qubits where no operations occur<br>
b. When all qubits are busy almost all the time (densely packed schedules)<br>
c. Only when running on a simulator with noise disabled<br>
d. Only when measuring expectation values with Estimator<br>


:::{dropdown}answer 
The answer is a. 

DD is primarily beneficial when there are idle periods; pulses inserted during idle intervals can suppress coherent errors. With densely packed schedules, DD can provide little benefit or even hurt due to pulse imperfections. 
:::


4. Which error‑related options are supported directly by Sampler V2?

a. Pauli twirling (suppression) via `sampler.options.twirling.*`<br>
b. TREX measurement mitigation via `sampler.options.resilience.measure_mitigation = True`<br>
c. Zero‑noise extrapolation via `sampler.options.resilience.zne_mitigation = True`<br>
d. Probabilistic error cancellation via `sampler.options.resilience.pec_mitigation = True`<br>


:::{dropdown}answer 
The answer is a. 

Sampler supports suppression methods like Pauli twirling and dynamical decoupling. Mitigation techniques such as TREX, ZNE, and PEC are available in Estimator; Sampler does not support them directly. 
:::


5. Which snippet correctly configures Pauli twirling in Sampler V2 with 32 randomizations and 100 shots per randomization?

a. 
```
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = 32
sampler.options.twirling.shots_per_randomization = 100
```

b. 
```
sampler.options.twirling.enable = True
sampler.options.twirling.count = 32
sampler.options.twirling.shots = 100
```

c. 
```
sampler.options.resilience.twirling = True
sampler.options.resilience.num_randomizations = 32
sampler.options.resilience.shots_per_randomization = 100
```

d. 
```
sampler.enable_twirling(32, 100)
```

:::{dropdown}answer 
The answer is a. 

Use the `twirling` options on Sampler: set `enable_gates=True`, then configure `num_randomizations` and `shots_per_randomization`. 
:::


6. How do you set a repetition delay of 0.5 ms between shots for Sampler V2?

a. sampler.options.execution.rep_delay = 0.0005<br>
b. sampler.options.environment.rep_delay = 0.5<br>
c. sampler.options.execution.rep_delay = 0.5<br>
d. sampler.set_options(rep_delay_ms=0.5)<br>


:::{dropdown}answer 
The answer is a. 

`rep_delay` is specified in seconds within the `execution` options; 0.5 ms equals 0.0005 seconds. 
:::


7. What is the default DD sequence used if you only enable `sampler.options.dynamical_decoupling.enable = True` and do not set `sequence_type`?

a. "XX"<br>
b. "XpXm"<br>
c. "XY4"<br>
d. There is no default; you must choose explicitly<br>


:::{dropdown}answer 
The answer is a. 

The default dynamical decoupling sequence is `"XX"` unless you override it. 
:::


8. Which statement about dynamical decoupling is TRUE?

a. Each inserted DD pulse sequence ideally implements an identity operation while suppressing coherent errors on idling qubits<br>
b. DD is a measurement‑error mitigation method that post‑processes bitstrings<br>
c. DD is guaranteed to improve results in all cases, even with dense schedules<br>
d. DD can only be used with Estimator, not with Sampler<br>


:::{dropdown}answer 
The answer is a. 

DD inserts carefully chosen pulse sequences during idle gaps. These sequences act as identity operations but help cancel coherent errors. It is a suppression (control) technique, not a post‑processing mitigation method. 
:::


## TASK 5.2: Understand the theoretical background behind the sampler primitive

1. What is the primary role of the **Sampler primitive** in Qiskit Runtime?

a. To estimate expectation values of observables from quantum circuits<br>
b. To sample bitstrings from quantum circuits and return probability distributions<br>
c. To automatically compile and optimize quantum circuits into hardware basis gates<br>
d. To implement error correction by dynamically repeating measurements<br>

:::{dropdown}answer
The answer is b.

The Sampler primitive is designed to run quantum circuits and return **quasi-probability distributions** of measurement outcomes (bitstrings). It does not calculate expectation values (that is the Estimator’s role), nor does it perform compilation or error correction itself.
:::

2. Which of the following best describes the **output** of the Sampler primitive?

a. A histogram of expectation values for given observables<br>
b. A quasi-probability distribution over bitstrings measured from the circuit<br>
c. A sequence of transpiled circuits optimized for the backend<br>
d. A set of error-mitigated observables in operator form<br>

:::{dropdown}answer
The answer is b.

The Sampler outputs a **quasi-probability distribution** that summarizes how often each bitstring is expected to occur. This distribution captures sampling statistics, while expectation values belong to Estimator results.
:::

3. From a theoretical perspective, how does the **Sampler primitive** relate to projective measurements in quantum mechanics?

a. It implements an approximation of eigenvalue estimation for arbitrary operators<br>
b. It directly samples computational basis states according to Born’s rule<br>
c. It generates a classical approximation of wavefunction amplitudes<br>
d. It collapses the state into eigenstates of Pauli operators only<br>

:::{dropdown}answer
The answer is b.

The Sampler models **Born’s rule**: after a quantum circuit is executed, measurements in the computational basis yield outcomes with probabilities proportional to the squared amplitudes of the final state vector. Sampler automates this sampling process.
:::

4. Which scenario is most appropriate for using the **Sampler primitive** instead of the Estimator?

a. When one needs to calculate expectation values of a Hamiltonian<br>
b. When one needs raw measurement outcomes or probability distributions from circuits<br>
c. When one needs to optimize circuit depth before hardware execution<br>
d. When one needs to compute gradients of parameterized circuits<br>

:::{dropdown}answer
The answer is b.

The Sampler is suited for tasks where **raw measurement statistics** (bitstring distributions) are needed, such as training a variational algorithm using measured samples. For Hamiltonian expectation values, the Estimator is the correct primitive.
:::

5. What theoretical advantage does using the **Sampler primitive** provide over manually executing circuits with `backend.run()`?

a. It provides guaranteed error correction of results<br>
b. It automatically transforms any observable into Pauli terms<br>
c. It offers a higher-level abstraction focused on probability distributions rather than low-level job control<br>
d. It bypasses the need for quantum circuit transpilation<br>

:::{dropdown}answer
The answer is c.

The Sampler simplifies the user workflow by exposing a **high-level interface** that directly provides probability distributions of bitstrings. Unlike `backend.run()`, it hides low-level details such as job management, allowing developers to focus on sampling tasks.
:::